## Blood Flow Velocity Recovery with Physics-informed Neural Networks (PINN)
\\
Complete ##TODOs and make sure your code generate the correct result



In [ ]:
### if you are using Google Colab

# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/MyDrive/pinn')

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import math
import os
import gc
import time
import random
# plotting
import matplotlib.pylab as plt
import matplotlib.ticker as ticker
# system
from time import time
import sys
# torch import
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim


### Prepare Dataset

In [ ]:
### This function is to help you understand the meanings of variables in the dataset and load x y coordinates for training.
def dataloader():

		train_size = 64  ### tune train_size if you like
		Data = np.load('xyuvp.npz')  ### full data
		Data_sparse = np.load('xyuvp_sparse_separate_3sec.npz') ### data file contains sparse sampled data
		### make sure you know what variables mean below. You will need to build inlet and outlet by yourself
		### These names should be straightforward
		xinlet = Data_sparse['xinlet']
		yinlet = Data_sparse['yinlet']
		uinlet = Data_sparse['uinlet'] ### velocity at inlet

		xoutlet = Data_sparse['xoutlet']
		youtlet = Data_sparse['youtlet']
		uoutlet = Data_sparse['uoutlet'] ### velocity at outlet
		x = Data['x']
		y = Data['y']
		u = Data['u']
		v = Data['v']
		P = Data['p']

		x = x[...,None]
		y = y[...,None]
		u = u[...,None]
		v = v[...,None]
		P = P[...,None]

		xb = Data_sparse['xb']   ### boundary of x
		yb = Data_sparse['yb']
		ub = Data_sparse['ub']
		vb = Data_sparse['vb']
		pb = Data_sparse['pb']



		sp_xdom = Data_sparse['xdom']   ### sparse sampled data
		sp_ydom = Data_sparse['ydom']
		sp_udom = Data_sparse['udom']   ### sparse sampled data
		sp_vdom = Data_sparse['vdom']
		sp_Pdom = Data_sparse['pdom']


		
		plt.figure(figsize=(12,8))
		plt.scatter(x,y, c=u, cmap = 'coolwarm')
		plt.colorbar()
		plt.scatter(sp_xdom,sp_ydom, marker='x', c='black') ### plot sparse sampled data
		plt.scatter(xb,yb, marker='x', c='orange') ### plot boundary data
		plt.axis('equal')
		plt.show()
		############################

		np.savez('stenosis_hard_coord',x = x,y = y,u = u, v = v, P = P)
		################
		data = torch.utils.data.TensorDataset(torch.FloatTensor(x), torch.FloatTensor(y))
		
		train_loader = torch.utils.data.DataLoader(data, batch_size=train_size, shuffle=True)
		print('len(data is)', len(data))
		print('len(dataloader is)', len(train_loader))
		return train_loader,train_size


In [ ]:
train_loader, train_size = dataloader()

### Build Neural Networks (30 Points)

In [ ]:
class Swish(nn.Module):
        def __init__(self, inplace=True):
            super(Swish, self).__init__()
            self.inplace = inplace

        def forward(self, x):
            if self.inplace:
                x.mul_(torch.sigmoid(x))
                return x
            else:
                return x * torch.sigmoid(x)
class BloodNet(torch.nn.Module):
    def __init__(self, n_feature, n_hidden):
        super(BloodNet, self).__init__()
        self.nu = 1e-3 ### viscosity
        self.rho = 1 ### density


        ### TODO
        ''' Build a five-layer Feedforward Neural Networks
        Refer to: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
        Often used Non-linearities: Swish(provided above), ReLU, Softmax, Sigmoid, Tanh...
        Inputs: 2D tensor x and y
        Outputs: 3D tensor u, v, P'''


        
        
        
    def forward(self, x):
      ### TODO
      '''Run x through the neural networks. Output u, v, P'''

In [ ]:
model = BloodNet(2,150)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)

In [ ]:
### Optional Initialization
def init_normal(m):
		if type(m) == nn.Linear:
			nn.init.kaiming_normal_(m.weight)
model.apply(init_normal)

### Define Loss Functions (20 Points)



In [ ]:
def criterion(model, x,y):


    x = torch.FloatTensor(x).to(device)
    y = torch.FloatTensor(y).to(device)
    x.requires_grad = True
    y.requires_grad = True




    net_in = torch.cat((x,y),1)

    output = model.forward(net_in)
    u = output[:,0]
    v = output[:,1]
    P = output[:,2]
    u = u.view(len(u),-1)
    v = v.view(len(v),-1)
    P = P.view(len(P),-1)





    ###Compute partial derivatives
    u_x = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]   ### du/dx
    u_xx = torch.autograd.grad(u_x,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]   
    u_y = torch.autograd.grad(u,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
    u_yy = torch.autograd.grad(u_y,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
    v_x = torch.autograd.grad(v,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
    v_xx = torch.autograd.grad(v_x,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
    v_y = torch.autograd.grad(v,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
    v_yy = torch.autograd.grad(v_y,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]

    P_x = torch.autograd.grad(P,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
    P_y = torch.autograd.grad(P,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
    ### TODO
    '''Implement the equation loss using partial derivatives computed above
    X-direction:
    Y-direction: 
    Continuity:  '''



    
    loss_f = nn.MSELoss()

    #Note our target is zero. It is residual so we use zeros_like
    loss = loss_f(loss_1,torch.zeros_like(loss_1))+  loss_f(loss_2,torch.zeros_like(loss_2))+  loss_f(loss_3,torch.zeros_like(loss_3))


    return loss
def Loss_BC(model,xb,yb, inlet_input, outlet_input, inlet_target, outlet_target  ):

    net_in1 = torch.cat((xb, yb), 1)
    output = model.forward(net_in1)
    u = output[:,0]
    v = output[:,1]  
    u = u.view(len(u),-1)
    v = v.view(len(v),-1)
    loss_f = nn.MSELoss()
    loss_noslip = loss_f(u, torch.zeros_like(u))


    inlet_out = model.forward(inlet_input)
    outlet_out = model.forward(outlet_input)
    loss_ic = loss_f(inlet_out[:,0], inlet_target[:,0]) + loss_f(outlet_out[:,0], outlet_target[:,0])


    return loss_noslip+loss_ic




def Loss_data(model,xd,yd,ud,vd):

    net_in1 = torch.cat((xd, yd), 1)
    output = model.forward(net_in1)
    u = output[:,0]
    v = output[:,1]
    p = output[:,2]
    
    u = u.view(len(u),-1)
    v = v.view(len(v),-1)
    p = p.view(len(p),-1)
    
    loss_f = nn.MSELoss()
    loss_d = loss_f(u, ud) 


    return loss_d





### Training Neural Networks (50 Points)

Evaluation Criterion: 
1. Plot of PINN prediction is similar to CFD simulation
2. loss_data is smaller than 1e-3

In [ ]:
import time

tic = time.time()
### Load Sparse Data and Boundary Data
Data_sparse = np.load('xyuvp_sparse_separate_3sec.npz')

sp_xin = Data_sparse['xinlet']
sp_yin = Data_sparse['yinlet']
sp_uin = Data_sparse['uinlet']
sp_xout = Data_sparse['xoutlet']
sp_yout = Data_sparse['youtlet']
sp_uout = Data_sparse['uoutlet']
sp_xdom = Data_sparse['xdom']
sp_ydom = Data_sparse['ydom']
sp_udom = Data_sparse['udom']
sp_vdom = Data_sparse['vdom']
sp_Pdom = Data_sparse['pdom']
sp_xb = Data_sparse['xb']
sp_yb = Data_sparse['yb']
sp_ub = Data_sparse['ub']
sp_vb = Data_sparse['vb']
sp_Pb = Data_sparse['pb']


sp_x = np.concatenate((sp_xdom,sp_xb),0)
sp_y = np.concatenate((sp_ydom,sp_yb),0)
sp_u = np.concatenate((sp_udom,sp_ub),0)
sp_v = np.concatenate((sp_vdom,sp_vb),0)
sp_P = np.concatenate((sp_Pdom,sp_Pb),0)
sp_x, sp_y, sp_u, sp_v, sp_P = sp_x[...,None], sp_y[...,None], sp_u[...,None], sp_v[...,None], sp_P[...,None]
sp_data = np.concatenate((sp_x,sp_y,sp_u,sp_v,sp_P),1)

# for sparase stenosis
sp_x, sp_y, sp_u, sp_v, sp_P = torch.Tensor(sp_data[:,0]).to(device), torch.Tensor(sp_data[:,1]).to(device), torch.Tensor(sp_data[:,2]).to(device), torch.Tensor(sp_data[:,3]).to(device), torch.Tensor(sp_data[:,4]).to(device)
sp_x, sp_y, sp_u, sp_v, sp_P = sp_x.view(len(sp_x), -1), sp_y.view(len(sp_y), -1), sp_u.view(len(sp_u), -1), sp_v.view(len(sp_v), -1), sp_P.view(len(sp_P),-1)

sp_x.reuqires_grad = True
sp_y.requires_grad = True



xb = torch.Tensor(sp_xb).to(device)
yb = torch.Tensor(sp_yb).to(device)
xb = xb.view(len(xb),-1)
yb = yb.view(len(yb),-1)
xb.requires_grad = True
yb.requires_grad = True
x_in = torch.Tensor(sp_xin).to(device)
y_in = torch.Tensor(sp_yin).to(device)
u_in = torch.Tensor(sp_uin).to(device)
x_in, y_in, u_in = x_in.view(len(x_in),-1), y_in.view(len(y_in),-1), u_in.view(len(u_in),-1)
x_in.requires_grad = True
y_in.requires_grad = True
inlet_input = torch.cat((x_in,y_in),1)
inlet_target = torch.cat((u_in,torch.zeros_like(u_in),torch.zeros_like(u_in)),1)


x_out = torch.Tensor(sp_xout).to(device)
y_out = torch.Tensor(sp_yout).to(device)
u_out = torch.Tensor(sp_uout).to(device)
x_out, y_out, u_out = x_out.view(len(x_out),-1), y_out.view(len(y_out),-1), u_out.view(len(u_out),-1)

x_out.requires_grad = True
y_out.requires_grad = True
outlet_input = torch.cat((x_out, y_out), 1)
outlet_target = torch.cat((u_out, torch.zeros_like(u_out), torch.zeros_like(u_out)), 1)
tol = 0
err = np.inf

'''Change epo to change how many epochs to run! You can start with 2000, but sometimes 1000 is enough if you are lucky.
   Monitor the training process and stop if you think the result looks similar with CFD simulation '''
epo = 2000


for epoch in range(epo):

    loss_eqn_tot = 0.
    loss_bc_tot = 0.
    loss_data_tot = 0.
    loss_tot = 0
    n = 0
    

    for batch_idx, (x,y) in enumerate(train_loader): 
        ### TODO
        '''Train the neural networks using the five steps we learned in class
           1. Forward the data through the model;
           2. Compute the loss (measure of fitting capability); call the loss functions defined above
           3. Zero the grad-parameters (partial derivative of the loss wrt the parameters);
           4. Accumulate the grad-parameters;
           5. Step in the opposite direction of the grad-parameters. '''
        
        
        
       
        
        loss_eqn_tot += loss_eqn
        loss_bc_tot += loss_bc
        loss_data_tot  += loss_data
        loss_tot += loss

    scheduler.step()
    if epoch % 100 ==0:
      
					Data = np.load('stenosis_hard_coord.npz')
					x = Data['x']
					y = Data['y']
					u_CFD = Data['u']

					
					xt,yt = torch.Tensor(x), torch.Tensor(y)
					xt,yt = xt.view(len(xt),-1), yt.view(len(yt),-1)
					xt.requires_grad = True
					yt.requires_grad = True
					xt, yt = xt.to(device), yt.to(device)
					inputs = torch.cat((xt,yt),1)
					with torch.no_grad():
							out_all = model.forward(inputs)
							u_hard = out_all[:,0]
							v_hard = out_all[:,1]
							P_hard = out_all[:,2]
							u_hard = u_hard.view(len(u_hard),-1)
							v_hard = v_hard.view(len(v_hard),-1)
							P_hard = P_hard.view(len(P_hard),-1)
							u_hard = u_hard.cpu().detach().numpy()
							v_hard = v_hard.cpu().detach().numpy()
							P_hard = P_hard.cpu().detach().numpy()
					

					plot_x= 0.4*np.max(x)
					plot_y = 0.95*np.max(y)
					fontsize = 18
					

					plt.figure()
					plt.subplot(2,1,1)
					plt.scatter(x, y, c= u_hard, label = 'uhard', cmap = 'coolwarm', vmin = min(u_CFD), vmax = max(u_CFD))
					plt.text(plot_x, plot_y, r'epoch {}'.format(epoch), {'color': 'b', 'fontsize': fontsize})
					plt.colorbar()

					plt.savefig('./results/{}.png'.format(epoch),bbox_inches = 'tight')


    print('*****Train Epoch: {} Total Loss {:.10f} Loss eqn {:.10f} Loss BC {:.10f} Loss data {:.10f} ****'.format(epoch, loss_tot, loss_eqn_tot, loss_bc_tot,loss_data_tot) )
torch.save(model.state_dict(), 'pinn_stenosis.pt')
f'finished in {(time.time()-tic)/60}'